In [47]:


import io

class Mat:
  """Implementação de uma matriz
  """
  def __init__(self, rows, cols):                                         
      """Construtor da classe

      Args:
          rows (int): _linhas da matrz
          cols (int): _colunas da matriz
      """
      self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]

  def numRows (self): return len(self.mat)

  def numCols (self): return len(self.mat[0])

  def __str__(self):
        """"Devolve a matriz como uma string
        """
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    
  def __repr__(self):
        return str(self)
    
  def __getitem__ (self, n):
        """Interface para a indexação []
        """
        return self.mat[n]


def subst(x,y):
  """Função que permite substituir valores de match e mismatch no alinhamento local

  Args:
    x (str): elemento da s1
    y (str): elemento da s2 
  Returns:
    int: valores de match(+4) e mismatch(-2)
    """  
                                                        
  return 4 if x == y else -2                                          

class NW:
  """Alinhamento global entre duas sequências usando valores de match e mismatch pré defenidos

  """
  def __init__(self, s1, s2, g=-2):
    """Contrutor da classe NW

    Args:
        s1 (str): 1ª sequência
        s2 (str): 2ª sequência
        g (int): Gap Penalty
    """
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
  
  def align_score(self):
    """ Função que recebe os alinhamentos

    Retorna:
        int: O score máximo do alinhamento NW entre duas sequências 
    """
    return self.mat[len(self.s1)][len(self.s2)]

  def rebuild(self):
    """Função que implementa o processo de reconstrução e recuperação do 
    alinhamento ótimo entre duas sequências

    Retorna:
        str: Alinhamento Ótimo entre as duas sequências
    """
                                              
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),               # Diagonal
        'E' : ( 0, -1),               # Esquerda
        'C' : (-1,  0)                # Cima
    }

    while L > 0 or C > 0:
      DL, DC = dirs[self.tr[L][C]]


      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        

      L += DL
      C += DC

    return S1, S2

  def __repr__(self):
    """Função que premite ver a representação das matrizes S e T

    Retorna:
        _type_: Matriz S e Matriz T
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print('\n Matriz T :', file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()


In [52]:
def teste_global_align():
    align = NW("ACHDHGT","THHDACCGT")
    print(align)
    print("\n Score do Alinhamento:", align.align_score())
    print("\n Alinhamento ótimo:",align.rebuild())  
teste_global_align()


In [26]:
align = NW("ACHDHGT","THHDACCGT",-2)
("\n Alinhamento ótimo:",align.rebuild()) 

D
D
D
E
E
D
D
D
D


('\n Alinhamento ótimo:', ('ACHD--HGT', 'THHDACCGT'))

In [41]:
import io

class Mat:

  def __init__(self, rows, cols):
        """Implementação de uma matriz
        """                                          
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]

  def numRows (self): return len(self.mat)

  def numCols (self): return len(self.mat[0])

  def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)

  def __repr__(self):
        return str(self)
        
  def __getitem__ (self, n):
        return self.mat[n]


def subst(x,y):
  """Função que permite substituir valores de match e mismatch no alinhamento local

  Args:
    x (str): elemento da s1
    y (str): elemento da s2 
  Returns:
    int: valores de match(+4) e mismatch(-2)
    """                                                         
  return 4 if x == y else -2
                                        
class SW:
  """Alinhamento global entre duas sequências usando valores de match e mismatch pré defenidos

  """
  def __init__(self, s1, s2, g):
    """Contrutor da classe SW

    Args:
        s1 (str): 1ª sequência
        s2 (str): 2ª sequência
        g (int): Gap Penalty
    """
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   
            self.mat[L+1][C    ] + g,               
            self.mat[L  ][C + 1] + g,               
            0]
        dirs = "DEC^"

        self.mat[L + 1][C + 1] = max(possiveis)
        if self.mat[L + 1][C + 1] != 0:
          self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
  

  def align_score(self): 
    """ Função que recebe os alinhamentos

    Retorna:
        int: O score máximo do alinhamento SW entre duas sequências 
    """
    align_score = 0
    row = 0
    column = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > align_score:
          align_score = max(x1)
          row = L
        column = C
    column -= 1
    return [align_score,row,column]

  def rebuild(self):
    """Função que implementa o processo de reconstrução (começando pelo canto inferior direito) e recuperação do 
    alinhamento ótimo entre duas sequências

    Retorna:
        str: Alinhamento Ótimo entre as duas sequências
    """                                          
    align_score = 0
    row = 0
    column = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > align_score:
          align_score = max(x1)
          row = L
        column = C
    column -= 1
    L  = row
    C  = column
    S1 = ""
    S2 = ""   
    
    dirs = {
        'D' : (-1, -1),               
        'E' : ( 0, -1),               
        'C' : (-1,  0),
        'O' : (0, 0)               
    }

    while L >= 0 or C >= 0:
      try:
        DL, DC = dirs[self.tr[L][C]]
        if self.tr[L][C] == "D":
          S1 = self.s1[L - 1] + S1
          S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "E":
          S1 = '-' + S1
          S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "C":
          S1 = self.s1[L - 1] + S1
          S2 = '-' + S2 

        L += DL
        C += DC
      except:
        break

    return S1, S2
  
  def __repr__(self):
    """Função que premite ver a representação das matrizes S e T

    Retorna:
        str: Matriz S e Matriz T
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print('\n Matriz T :', file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()


In [50]:
def teste_alignSW():
    align = SW("HAYFAFHSFF","FAGSTFS",-4)
    print(align)
    print("\n Score do Alinhamento:", align.align_score(), sep="\n")
    print("\n Alinhamento ótimo:",align.rebuild(), sep="\n")  
teste_alignSW()


 Matriz S :
    -   F   A   G   S   T   F   S
-   0   0   0   0   0   0   0   0
H   0   0   0   0   0   0   0   0
A   0   0   4   0   0   0   0   0
Y   0   0   0   2   0   0   0   0
F   0   4   0   0   0   0   4   0
A   0   0   8   4   0   0   0   2
F   0   4   4   6   2   0   4   0
H   0   0   2   2   4   0   0   2
S   0   0   0   0   6   2   0   4
F   0   4   0   0   2   4   6   2
F   0   4   2   0   0   0   8   4

 Matriz T :
  - F A G S T F S
- 0 0 0 0 0 0 0 0
H 0 0 0 0 0 0 0 0
A 0 0 D 0 0 0 0 0
Y 0 0 0 D 0 0 0 0
F 0 D 0 0 0 0 D 0
A 0 0 D E 0 0 0 D
F 0 D C D D 0 D 0
H 0 0 D D D 0 0 D
S 0 0 0 0 D D 0 D
F 0 D 0 0 C D D E
F 0 D D 0 0 0 D D


 Score do Alinhamento:
[8, 5, 9]

 Alinhamento ótimo:
('', '')


In [32]:
align = SW("ACGT","TACCGT",-4)
    
print("\n Score do Alinhamento:", align.align_score(), sep="\n")


 Score do Alinhamento:
[0, 0, 3]


In [ ]:
while L >= 0 or C >= 0:
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2  

      L += DL
      C += DC

    return S1, S2

In [ ]:

    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

In [14]:
import unittest

class Test_NW_ALIGNMENT(unittest.TestCase):
    def test_difsize(self):
        align = SW("ACGT","TACCGT",-4)
        score = align.align_score()
        self.assertEqual(score , 6)
  
        
        

    
if __name__ == '__main__':
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\joana\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-120281Ooq8zHG2Eka.json'


AssertionError: 